# Avaliação e validação de experimentos


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
# Semente aleatória para reproducibilidade dos experimentos (reproducão dos experimentos)

SEED = 17

In [ ]:
df_raw = pd.read_csv("data/bots_vs_users.csv")

df_raw.info()


In [ ]:
df_raw.T

In [ ]:

df_interm = df_raw[df_raw["has_domain"] != "Unknown"]

# Colunas com todos valores iguais
df_interm = df_interm.drop(columns=["has_domain", "has_short_name", "has_first_name", "has_last_name"])

# Colunas com muitos (> 4500) "Unknown"
df_interm = df_interm.drop(columns=["has_tv", "has_quotes", "has_about", "has_games", "has_movies", "has_activities", "has_music", "has_career", "has_military_service", "has_hometown", "marital_status", "has_universities", "has_schools", "has_relatives", "has_personal_data", "has_interests", "has_books"])

# Colunas com muitos (> 2500) "Unknown"
df_interm = df_interm.drop(columns=["has_nickname", "has_maiden_name", "has_mobile", "all_posts_visible", "audio_available", "can_add_as_friend", "can_invite_to_group", "is_blacklisted", "is_verified", "is_confirmed", "has_status", "city", "has_occupation", "occupation_type_university", "occupation_type_work", "subscribers_count"])

# Colunas com muitos (> 4400) NaN
df_interm = df_interm.drop(columns=["ads_ratio", "avg_likes", "links_ratio", "hashtags_ratio", "avg_views", "posting_frequency_days", "phone_numbers_ratio", "avg_text_uniqueness", "avg_keywords", "avg_text_length", "attachments_ratio", "avg_comments", "reposts_ratio", "posts_count"])

# Substitui os Unknown por zero e converte os numeros para int
df_interm = df_interm.replace("Unknown", 0).apply(pd.to_numeric, errors="ignore")

# Tirar algumas colunas aleatorias para dificultar
# df_interm = df_interm.drop(columns=[])

# for col in df.columns:
#     print(df[col].value_counts(dropna=False))
#     print('---')

df_interm.columns

In [ ]:
# Tirar uma amostra menor dos dados
df, _ = train_test_split(df_interm, train_size=1000, stratify=df_interm["target"], random_state=SEED)


X = df.drop(columns=['target'])

yreg = df.target
ycla = yreg > 0

print(X.shape)

## Classificação

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import make_scorer

In [ ]:
scorer = make_scorer(accuracy_score) # Teste outras

cv = StratifiedKFold(n_splits=15, shuffle=True, random_state=SEED)
gscv = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)

algorithms = {
    'kNN':  GridSearchCV(
        Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', MinMaxScaler(feature_range=(0, 1))),
            ('selector', VarianceThreshold()),
            ('knn', KNeighborsClassifier())]),
        param_grid={
            'selector__threshold': [0, 0.01, 0.02, 0.03],
            'knn__n_neighbors': [1, 3, 5],
            'knn__p': [1, 2],
        },
        scoring=scorer,
        cv=gscv),


    'tree':  GridSearchCV(
        Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('tree', DecisionTreeClassifier(random_state=SEED))]),
        param_grid={
            'tree__max_depth': [5, 10, 20],
            'tree__criterion': ['entropy', 'gini'],
        },
        scoring=scorer,
        cv=gscv),


    'bigtree':  GridSearchCV(
        Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('tree', DecisionTreeClassifier(max_depth=None, min_samples_split=2, random_state=SEED))]),
        param_grid={
            'tree__criterion': ['entropy', 'gini'],
        },
        scoring=scorer,
        cv=gscv),


    'nb': GridSearchCV(
        Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('selector', SelectKBest()),
            ('nb', GaussianNB())]),
        param_grid={
            'selector__k': [3, 5, 10],
        },
        scoring=scorer,
        cv=gscv),


    'svmlinear': GridSearchCV(
        Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('pca', PCA()),
            ('svm', SVC(kernel='linear', random_state=SEED))]),
        param_grid={
            'pca__n_components': [2, 5, 10],
            'svm__C': [1.0, 2.0],
        },
        scoring=scorer,
        cv=gscv),

        
    'svmrbf': GridSearchCV(
        Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler()),
            ('pca', PCA(random_state=SEED)),
            ('svm', SVC(kernel='rbf', random_state=SEED))]),
        param_grid={
            'pca__n_components': [2, 5, 10],
            'svm__C': [1.0, 2.0],
            'svm__gamma': [0.1, 1.0, 2.0],
        },
        scoring=scorer,
        cv=gscv),
}

In [ ]:
from sklearn.model_selection import cross_val_score

result = {}
for alg, clf in algorithms.items():
  result[alg] = cross_val_score(clf, X, ycla, cv=cv)

result = pd.DataFrame.from_dict(result)

In [ ]:
print(result)

In [ ]:
result.apply(lambda x: "{:.2f} ± {:.2f}".format(x.mean(), x.std()))

In [ ]:
import matplotlib.pyplot as plt

plt.boxplot([ scores for alg, scores in result.items()])
plt.xticks(1 + np.arange(result.shape[1]), result.columns)

plt.axhline(1, linestyle="dotted", color="green")
plt.ylim(0,1.05)  # Nao mentir com dados para os coleguinhas

plt.show()

## Comparar desempenho (validar hipótese)

In [ ]:
from scipy.stats import wilcoxon

In [ ]:
# Comparar dois algoritmos:
wilcoxon(result.kNN, result.tree)

In [ ]:
# Comparar todos contra todos: Friedman + Nemenyi (further reading), construir um diagrama de diferença crítica
# Orange3

## Melhor modelo (deploying)

In [ ]:
classifier = algorithms['bigtree']
classifier.fit(X, ycla) # Uso a base toda!
print(classifier.best_estimator_)

In [ ]:
x = X.iloc[0:1, :]
print(x) # Faz de conta que é novo

In [ ]:
classifier.predict(x)

### Um outro modelo

In [ ]:
classifier = algorithms['svmrbf']
classifier.fit(X, ycla) # Uso a base toda
print(classifier.best_estimator_)

In [ ]:
print(x)
x.iloc[0, 13] = np.nan
print(x)
classifier.predict(x)

## Regressão

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import make_scorer

In [ ]:
scorer = make_scorer(mean_squared_error, greater_is_better=False)
algorithms = {}